In [ ]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from emcee import EnsembleSampler
from multiprocessing import Pool
from corner import corner
from astropy.io import fits
from multiprocessing import Pool
from scipy import stats
from scipy import optimize
import numpy.random as random
import emcee
import corner
import random
from PyAstronomy import funcFit as fuf
from PyAstronomy import pyasl
from gaussfold import gaussfold as gf
from IPython.display import display, Math
from specutils.fitting import fit_continuum
from specutils.spectra import Spectrum1D, SpectralRegion
from specutils.fitting import fit_generic_continuum
from astropy.modeling import models
import multiprocessing as mp
Pool = mp.get_context('fork').Pool
from dust_extinction.parameter_averages import F19
# define the model
ext = F19(Rv=3.1)

In [ ]:
igrinswave = np.array(igwave[index,:])
igrinsspec = np.array(igspec_diva0[index,:])
igrinserr = np.array(igrinsspec*fracerr[index,:])
ll = np.where((np.isfinite(igrinserr))&(np.isfinite(igrinswave))&(np.isfinite(igrinsspec)))
igrinsspec = np.squeeze(igrinsspec[ll])
igrinserr = np.squeeze(igrinserr[ll])
igrinswave = np.squeeze(igrinswave[ll])
ll = np.where(igrinswave > 16955)
print(np.shape(igrinswave),np.shape(ll))
igrinsspec = igrinsspec[ll]
igrinserr = igrinserr[ll]
igrinswave = igrinswave[ll]
igrinserr/=np.median(igrinsspec[np.where((igrinswave > wrange[0])&(igrinswave < wrange[1]))])
igrinsspec/=np.median(igrinsspec[np.where((igrinswave > wrange[0])&(igrinswave < wrange[1]))])
plt.plot(igrinswave+2,igrinsspec,label='data')

modelpath = '/Users/andrewmann/Dropbox/Radii/Models_K233_Feb2020.fits'
hdul = fits.open(modelpath)
hdul.info()
hdr = hdul[1].header
data = hdul[1].data
lambda0 = 4000
nlambda = 210001
dlambda = 0.1
modelwave = np.linspace(lambda0,lambda0+nlambda*dlambda,num=nlambda)
modelteff = np.squeeze(data['teff'])
modellogg = np.squeeze(data['logg'])
modelafe = np.squeeze(data['a_fe'])
modelmh = np.squeeze(data['metal'])
modelspectra = np.squeeze(data['spectrum'])

## resample model spectra
from gaussfold import gaussfold as gf
newlambda = np.array(igrinswave)
newmodel = np.zeros((np.size(newlambda),np.size(modelteff)))
for i in range(0,np.size(modelteff)):
    tmp = np.array(modelspectra[:,i])
    #print(np.shape(modelwave),np.shape(tmp))
    interpfunc = interpolate.interp1d(modelwave, tmp, kind='linear',fill_value="extrapolate")
    modelspec_new = interpfunc(newlambda)
    modelspec_new = gf(newlambda, modelspec_new, np.median(newlambda)/45000)
    newmodel[:,i] = modelspec_new

modelspectra = newmodel
modelwave = newlambda

modelspec = doublespec(modelteff, modelspectra, 3540,3520,0.01)
modelspec/=np.median(modelspec[np.where((modelwave > wrange[0])&(modelwave < wrange[1]))])
plt.plot(modelwave,modelspec,alpha=0.5,label='nospot')
modelspec2 = doublespec(modelteff, modelspectra, 3540,2800,0.7)
modelspec2/=np.median(modelspec2[np.where((modelwave > wrange[0])&(modelwave < wrange[1]))])
plt.plot(modelwave,modelspec2,alpha=0.5,label='spot')
plt.xlim(wrange[0],wrange[1])
plt.ylim(0.5,1.2)
plt.legend()
plt.show()

#plt.plot(modelwave,)

# x = np.arange(27)
# for ii in x:
#     print(ii)
#     igrinswave = igwave[ii,:]
#     ll = np.where((modelwave > np.min(igrinswave))&(modelwave < np.max(igrinswave)))
#     plt.plot(modelwave,modelspec2/np.median(modelspec2[ll]))
#     plt.plot(modelwave,modelspec/np.median(modelspec[ll]))
#     plt.xlim(np.min(igrinswave),np.max(igrinswave))
#     plt.ylim(0.3,1.4)
#     plt.show()

In [ ]:
## here let's make some example spectra to show off why this method works
#plt.plot(obswav_cut,obsspec_cut/np.median(obsspec_cut),label='data')
spec1 = doublespec(modelteff,modelspectra,3540,2900,0.2)
plt.plot(obswav_cut,spec1/np.median(spec1),label='20% spots')
spec2 = doublespec(modelteff,modelspectra,3540,2900,0.3)
plt.plot(obswav_cut,spec3/np.median(spec3),label='50% spots')
spec4 = doublespec(modelteff,modelspectra,3540,2900,0.7)
plt.plot(obswav_cut,spec4/np.median(spec4),label='70% spots')


fig, ax = plt.subplots()
fig.set_size_inches(7, 3.5)

plt.tight_layout
plt.xlim(6000,20000)
#plt.xscale('log')
#plt.yscale('log')
plt.legend()
plt.show()
